In [1]:
!pip install tensorflow

     |████████████████████████████████| 454.4 MB 117 kB/s eta 0:00:012     |████████████████████████▌       | 348.2 MB 8.4 MB/s eta 0:00:13
     |████████████████████████████████| 1.0 MB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 8.4 MB/s  eta 0:00:01
     |████████████████████████████████| 4.4 MB 3.5 MB/s eta 0:00:01
     |████████████████████████████████| 6.0 MB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 132 kB 506 kB/s eta 0:00:01
     |████████████████████████████████| 14.9 MB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 4.8 MB/s  eta 0:00:01
     |████████████████████████████████| 4.0 MB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 462 kB 3.9 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 1.5 MB/s eta 0:00:011
     |████████████████████████████████| 147 kB 2.5 MB/s eta 0:00:01
     |████████████████████████████████| 97 

In [1]:
!pip install numpy scipy matplotlib librosa

     |████████████████████████████████| 15.4 MB 10.5 MB/s eta 0:00:01
     |████████████████████████████████| 27.2 MB 6.4 MB/s eta 0:00:011
     |████████████████████████████████| 10.3 MB 16.8 MB/s eta 0:00:01    |██████████████████████████████▉ | 9.9 MB 16.8 MB/s eta 0:00:01
     |████████████████████████████████| 203 kB 16.8 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 26.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 30.2 MB/s eta 0:00:01
     |████████████████████████████████| 51 kB 1.9 MB/s  eta 0:00:01
     |████████████████████████████████| 377 kB 37.4 MB/s eta 0:00:01
     |████████████████████████████████| 303 kB 27.0 MB/s eta 0:00:01
     |████████████████████████████████| 24.9 MB 10.4 MB/s eta 0:00:01
     |████████████████████████████████| 3.4 MB 15.2 MB/s eta 0:00:01
     |████████████████████████████████| 323 kB 15.8 MB/s eta 0:00:01
     |████████████████████████████████| 25.3 MB 9.8 MB/s eta 0:00:011
     |██████████████████████████

In [2]:
import numpy as np
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
from scipy import fft
import os

In [2]:
spoof_source = "LA"
type_of_data = "dev"
generate_files_at = "../Data/ASVSpoof2019/"

desired_sampling_rate = 16000 # in Hz
time_duration = 4 # in seconds

window_size = 108 #in ms
window_shift = 10 #in ms
pointFFT = 1728

push_size = 2000 # how many samples to push into csv file at a time

In [3]:
datapath = "../Data/ASVSpoof2019/" + spoof_source + "/ASVspoof2019_" + spoof_source + "_" + type_of_data + "/flac"
labelfile = "../Data/ASVSpoof2019/" + spoof_source + "/ASVspoof2019_" + spoof_source + \
            "_cm_protocols/ASVspoof2019." + spoof_source + ".cm." + type_of_data + ".trl.txt"
window_size = int(window_size*desired_sampling_rate/1000) # converting to samples
window_shift = int(window_shift*desired_sampling_rate/1000) # converting to samples

In [4]:
import warnings
warnings.filterwarnings("error")

bonafide_files = []
spoof_files = []
count = 0
count0 = 0
count1 = 0
with open(labelfile) as fp:
    for line in fp:
        contents = line.split(' ')
        audioname = contents[1] + '.flac'
        if contents[4] == 'bonafide\n': # bonafide
            bonafide_files.append(audioname) # 0 for bonaifde, aka not fake
            count0 += 1
        else: #spoofed
            spoof_files.append(audioname) # 1 for spoofed, aka fake
            count1 += 1
        count += 1
print("Total number of label entries found = ", count, len(bonafide_files)+len(spoof_files))
print("Total number of bonafide / non-fake samples found = ", count0)
print("Total number of spoofed / fake samples found = ", count1)

signal, _ = librosa.load(os.path.join(datapath,bonafide_files[0]),sr = None)
while np.shape(signal)[0] < time_duration*desired_sampling_rate:
    signal = np.hstack((signal,signal))
signal = signal[:time_duration*desired_sampling_rate]
spectrogram = np.log(np.abs(librosa.stft(signal, n_fft = window_size, hop_length = window_shift)))
dim1 = np.shape(spectrogram)[0]
dim2 = np.shape(spectrogram)[1]

buffer_data = np.zeros((push_size, dim1*dim2), dtype = spectrogram.dtype)

bonafide_data_set = generate_files_at + spoof_source + "_" + type_of_data + "_bonafide.csv"
spoof_data_set = generate_files_at + spoof_source + "_" + type_of_data + "_spoof.csv"

if os.path.exists(bonafide_data_set):
    os.remove(bonafide_data_set)

count = 0
rem = 0
for filename in bonafide_files:
    # print(count, rem)
    fname= os.path.join(datapath,filename)
    signal, sr = librosa.load(fname,sr = None)
    while np.shape(signal)[0] < time_duration*desired_sampling_rate:
        signal = np.hstack((signal,signal))
    signal = signal[:time_duration*desired_sampling_rate]
    try:
        spectrogram = np.log(np.abs(librosa.stft(signal, n_fft = window_size, hop_length = window_shift)))
    except RuntimeWarning:
        print(count)
    buffer_data[rem, :] = spectrogram.reshape((1,dim1*dim2))
    
    count += 1
    rem+=1
    if rem == push_size:
        rem = 0
        print(count)
        with open(bonafide_data_set, "a") as f:
            np.savetxt(f, buffer_data, delimiter=",")

if rem > 0:
    with open(bonafide_data_set, "a") as f:
        np.savetxt(f, buffer_data[:rem,:], delimiter=",")
        
        

# if os.path.exists(spoof_data_set):
#     os.remove(spoof_data_set)        
        
        
# count = 0
# rem = 0
# for filename in spoof_files:
#     fname= os.path.join(datapath,filename)
#     signal, sr = librosa.load(fname,sr = None)
#     while np.shape(signal)[0] < time_duration*desired_sampling_rate:
#         signal = np.hstack((signal,signal))
#     signal = signal[:time_duration*desired_sampling_rate]
#     try:
#         spectrogram = np.log(np.abs(librosa.stft(signal, n_fft = window_size, hop_length = window_shift)))
#     except RuntimeWarning:
#         print(count)
#     buffer_data[rem, :] = spectrogram.reshape((1,dim1*dim2))
    
#     count += 1
#     rem+=1
#     if rem == push_size:
#         rem = 0
#         print(count)
#         with open(spoof_data_set, "a") as f:
#             np.savetxt(f, buffer_data, delimiter=",")

# if rem > 0:
#     with open(spoof_data_set, "a") as f:
#         np.savetxt(f, buffer_data[:rem,:], delimiter=",")

Total number of label entries found =  24844 24844
Total number of bonafide / non-fake samples found =  2548
Total number of spoofed / fake samples found =  22296
2000


KeyboardInterrupt: 

In [7]:
spoof_data_set = generate_files_at + spoof_source + "_" + type_of_data + "_spoof.csv"
count = 0
with open(spoof_data_set) as fp:
    for line in fp:
        int(line.split(','))
        count+=1
        break
print(count)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

In [3]:
A = np.array([[1,2],[3,4]])
print(np.square(A))

[[ 1  4]
 [ 9 16]]
